# Changes made to dec.pl

In [ ]:
?- cd('~/work'), ['dec:notation'].

true.

This notebook details the changes I have made to the dec.pl file during the last ten weeks. (For changes made to the Deepnote kernel, see [using_jswipl.ipynb](using_jswipl.ipynb)).

General changes:
- Adding an agent / actor argument
- Derived fluents
- Added a check for existing `holdsAtCached` fluents in the `tick/1` predicate to avoid duplicate `holdsAtCached` assertions.
- Removed an obsolete clause declaring `holdsAtNoCache` fluents at T = -1.

`Eval` related changes:
- Provided support for the `or` operator in addition to the existing `and` operator
- Added a `condition/1` predicate which allows logical conditions to be checked
- Added to the `eventually`, `never`, `always` and `until` support in `eval` clauses, so that `within`, `withinNextStart`, `later`, `delay`, `before` and `preceded` can also be used.
- Added support for adjustable expectation countdown times, using `within` as a test case.
- Added support for expectations which are independent of the expectation rule which created them


### Actors

As discussed in the **AgentsAndEvents** notebook, predicates in **dec.pl** have been altered so that `happensAtNarrative`, `initially`, `initiates` and `terminates` predicates should have a first argument in the form `Agent:Event`, rather than simply `Event`. `HappensAt`, `holdsAt`, `holdsAtCached`, `releases`, `releasedAtCached`, `progress` and `label` take an additional Actor argument as the first parameter.

Fluents which hold and events which occur for one agent will not hold / occur for another agent unless explicitly stated, or inheritance rules are defined or the agent parameter is a free variable when the fluent / event is declared.

For example, we can declare that a `pressButton` event occurs for the `home` actor at time period 1. We can then check that the event and follow on fluent hold only for this actor:

In [ ]:
% File: actorExample.pl
happensAtNarrative(home:pressButton(kitchen), 1).
initiates(home:pressButton(Room), lightOn(Room), _).

In [ ]:
?- run(3).
?- happensAt(Actor, Event, T).
?- holdsAt(Actor, Fluent, 2).
?- holdsAt(another, Fluent, 2).
?- holdsAt(house, Fluent, 2).

true.
Actor = home, Event = pressButton(kitchen), T = 1 .
Actor = home, Fluent = lightOn(kitchen) .
false.
false.

In [ ]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
true.
true.

### Derived fluents

A **derived_fluent/1** predicate was introduced to support state constraints involving the same type of fluent in both the head and the body of the clause. This meant that strata operations were supported even if the fluent was not used in a `initially`, `initiates` or `terminates` clause. However, it no longer appears to be needed:

We declare some parent-child relationships to hold for the **familyTree** agent, and declare that parents are ancestors, and people who are the parents of ancestors are also ancestors (a state constraint involving `ancestor/2` in both the head and the body).

In [ ]:
% File: ancestors.pl
initiates(familyTree:addParent(Parent, Child), parent(Parent, Child),_).
happensAtNarrative(familyTree:addParent(anne, bob),1).
happensAtNarrative(familyTree:addParent(anne, charlotte),1).
happensAtNarrative(familyTree:addParent(bob, diana),1).
happensAtNarrative(familyTree:addParent(charlotte, ellie),1).

holdsAt(familyTree, ancestor(Parent, Child), T):- holdsAt(familyTree, parent(Parent, Child), T).
holdsAt(familyTree, ancestor(Elder, Younger), T):- holdsAt(familyTree, parent(Elder, Offspring),T), 
holdsAt(familyTree, ancestor(Offspring, Younger), T).

When we check the fluents which hold at a later time period, both our parent and reliant ancestor fluents hold correctly. We did not have to define `derived_fluent(ancestor/2)`.

In [ ]:
?- run(10).
?- holdsAt(familyTree, Fluent, 4).

true.
Fluent = parent(anne, bob) ;
Fluent = parent(anne, charlotte) ;
Fluent = parent(bob, diana) ;
Fluent = parent(charlotte, ellie) ;
Fluent = ancestor(anne, bob) ;
Fluent = ancestor(anne, charlotte) ;
Fluent = ancestor(bob, diana) ;
Fluent = ancestor(charlotte, ellie) ;
Fluent = ancestor(anne, diana) ;
Fluent = ancestor(anne, ellie) .

## Changes made to eval

In [ ]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
true.
true.

The `eval/4` predicate contains the following parameters: **Actor**, **Expression**, **Time period**, **Boolean result**.
While `eval/4` in its orginal form could be used to check an event happened at or a fluent held at a particular time, it was not possible to evaluate a logical expression using this predicate.

Below, we declare an example event, `pressButton(kitchen)` which occurs at time one for the agent `home`, and which initiates the fluent `lightOn(kitchen)`. We can then check whether these and other events happen or fluents hold at a given time period using `eval`.

In [ ]:
% File: evalTest.pl
happensAtNarrative(home:pressButton(kitchen), 1).
initiates(home:pressButton(Room), lightOn(Room), _).

Note that if you are using multi-valued fluents, you should wrap them in brackets so that preference is applied to the colon when checking matching predicates. Here, we declare a carpet multi-valued fluent in brackets and a wallpaper fluent not wrapped in brackets.

In [ ]:
% File: initiallyTest.pl
initially(home:wallPaper="Flowers").
initially(home:(carpet="Green")).

The events and fluents declared above correctly evaluate to true, while undefined events and fluents evaluate to false.

In [ ]:
?- run(50).
?- eval(home, happ(pressButton(Room)), 1, Boolean).
?- eval(home, happ(handStand), 1, Boolean).
?- eval(home, lightOn(Room), 2, Boolean).
?- eval(home, lightOff(Room), 2, Boolean).
?- eval(house, happ(pressButton(Room)), 1, Boolean).

true.
Room = kitchen, Boolean = true .
Boolean = false .
Room = kitchen, Boolean = true .
Room = _1594, Boolean = false .
Room = _1596, Boolean = false .

Note that the carpet multi-valued fluent holds correctly at T = 0, while the wallpaper multi-valued fluent, which was not nested in brackets, is not produced by an `initially` query with the agent specified, and thus does not hold at T = 0.

In [ ]:
?- initially(AgentWithFluent).
?- initially(home:Fluent).
?- holdsAt(home,Fluent, 0).

AgentWithFluent = {':(home, wallPaper)': b'Flowers'} ;
AgentWithFluent = :(home, =(carpet, b'Green')) .
Fluent = {'carpet': b'Green'} .
Fluent = {'carpet': b'Green'} .

### Logical conditions

A standard logical expression evaluates to false:

In [ ]:
?- eval(_, 1==1, _, Boolean).
?- eval(_, 2==1, _, Boolean).

Boolean = false .
Boolean = false .

However, the following line of code was added to dec.pl, which means that logical expressions wrapped in `condition()` can be successfully evaluated:

`eval(_, condition(X),_,Bool):- (X -> Bool = true ; Bool = false).`


In [ ]:
?- eval(_, condition(1==1), _, Boolean).
?- eval(_, condition(2==1), _, Boolean).

Boolean = true .
Boolean = false .

While an expression like `1 == 1` is not inherently very useful to check, adding support for the `condition` argument allows more logical conditions to be applied to an `eval` check. For example, we can check if the argument in our `pressButton` event is the kitchen:

In [ ]:
?- eval(home, and([happ(pressButton(Room)), condition(Room == kitchen)]), 1, Boolean).
?- eval(home, and([happ(pressButton(Room)), condition(Room \= kitchen)]), 1, Boolean).


Room = kitchen, Boolean = true .
Room = kitchen, Boolean = false .

### Or operator

We are also now able to use the `or` operator in addition to the `and` operator in our `eval/4` clauses. This required creating `reduce_or/3` in the same style as `reduce_and/3` and applying `map_eval` to the results.

In [ ]:
?- eval(_, and([condition(1==1), condition(2==1)]), _, Boolean).
?- eval(_, or([condition(1==1), condition(2==1)]), _, Boolean).
?- eval(_, or([condition(1==1), condition(2==2)]), _, Boolean).
?- eval(_, or([condition(0==1), condition(1==2)]), _, Boolean).

Boolean = false .
Boolean = true .
Boolean = true .
Boolean = false .

Like `and`, `or` takes a list of conditions as an argument. We can successfully nest `and` and `or` operations:

In [ ]:
?- eval(_, and([condition(1==1), or([condition(1==1)])]), _, Boolean).
?- eval(_, and([condition(1==1), or([condition(2==1), condition(1=1)])]), _, Boolean).

Boolean = true .
Boolean = true .

### Extra temporal eval clauses

The original version of dec.pl that I used had support for `next`, `until`, `never`, `always` and `eventually` conditions in `eval` statements, which meant that we could use these conditions in expectation rules. Now,`within`, `withinStartNext`, `later`, `delay`, `before` and `preceded` can also be used. Note that if the `preceded` clause checks for events occurring or fluents holding before T = 0, this will always produce a `false` result.

Currently, these clauses are set up to be fulfilled or violated at the end of the time range specified, rather than one time period later. For example, if we expect that an event B will occur within 3 time periods of event A, which occurs at time period 1, if event B does not occur at time period 1, 2, 3 or 4, the expectation will be violated at time period 4 rather than time period 5.

In order to exhibit these conditions, we reset out environment:

In [ ]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).


true.
true.
true.

To illustrate the use of these clauses, I will use a traffic light scenario:

- A "stop" signal initiates the colour fluent to red, while a "go" signal initiates it to green and a "slow" signal initiates it to orange.

The expectation rules declared are as follows:
- The traffic light will be green **within** 5 time periods of a stop event.
- The traffic light will be orange within 7 time periods of a stop event, not counting the colour of the light when the stop event occurs (**withinStartNext**).
- The traffic light will only get a "go" signal more than 2 time periods after a "stop" signal (expectation that a "go" signal will only occur **later** than 2 periods after the "stop" signal).
- The traffic light will get a "go" signal exactly 3 time periods after a "stop" signal (**delay** expectation)
- When a "stop" signal is received we expect that there will be a red light **before** an green light holds.
- When a "go" signal occurs, it will have been **preceded** by a "stop" signal 3 time periods earlier.

According to our narrative events, the traffic light receives a "stop" signal at time period one, a "go" signal at time period four and a "slow" signal at time period 7, and the light colour is initially orange. Note that as our `colour` fluent is multi-valued, it must be nested in brackets in our `initially` clause setting it to orange.

In [ ]:
% File: trafficLightExample.pl
initiates(trafficLight:stop, colour=red, _).
initiates(trafficLight:go, colour=green, _).
initiates(trafficLight:slow, colour=orange, _).

initially(trafficLight:exp_rule(happ(stop), within(colour=green, 5), dependent, 
"Traffic light will be green within 5 time periods of a stop event")).
initially(trafficLight:exp_rule(happ(stop), withinStartNext(colour=orange, 7), dependent,
"Traffic light will be orange within 7 time periods of a stop event, not checking the current time period")).
initially(trafficLight:exp_rule(happ(stop), later(happ(go), 2), dependent,
"Traffic light will get go signal more than 2 time periods after stop signal")).
initially(trafficLight:exp_rule(happ(stop), delay(happ(go), 3), dependent, 
"Traffic light will get go signal exactly 3 time periods after stop signal")).
initially(trafficLight:exp_rule(happ(stop), before(colour=red, colour=green), dependent, 
"On stop signal, red light will happen before green light")).

initially(trafficLight:exp_rule(happ(go), preceded(happ(stop),3), dependent,
"On go signal, stop signal will have occurred 3 time periods earlier")).

initially(trafficLight:(colour=orange)).
happensAtNarrative(trafficLight:stop, 1).
happensAtNarrative(trafficLight:go, 4).
happensAtNarrative(trafficLight:slow, 7).

In [ ]:
?- run(15).

true.

The **stop** event at time period one triggers five expectations. Note that although the traffic light is currently orange, it does not fulfil our expectation that the traffic light will be orange within 7 time periods of the stop event, as we do not check the current time period.

In [ ]:
?- happensAt(trafficLight, Event, 1).
?- holdsAt(trafficLight, colour=Colour, 1).
?- holdsAt(trafficLight, exp(_, _, _, Outcome, _, Message), 1).


Event = stop .
Colour = orange .
Outcome = within(=(colour, green), 5), Message = b'Traffic light will be green within 5 time periods of a stop event' ;
Outcome = withinStartNext(=(colour, orange), 7), Message = b'Traffic light will be orange within 7 time periods of a stop event, not checking the current time period' ;
Outcome = later(happ(go), 2), Message = b'Traffic light will get go signal more than 2 time periods after stop signal' ;
Outcome = delay(happ(go), 3), Message = b'Traffic light will get go signal exactly 3 time periods after stop signal' ;
Outcome = before(=(colour, red), =(colour, green)), Message = b'On stop signal, red light will happen before green light' .

At T = 2, and T = 3, the countdown value for our `within`, `later` and `delay` expectations decrease by one each time. 

Note that at T = 2, our `withinStartNext` expectation has become a `within` expectation.

In [ ]:
?- T = 2, holdsAt(trafficLight, exp(_, _, _, Outcome, _, Message), T).
?- T = 3, holdsAt(trafficLight, exp(_, _, _, Outcome, _, Message), T).

T = 2, Outcome = within(=(colour, green), 4), Message = b'Traffic light will be green within 5 time periods of a stop event' ;
T = 2, Outcome = within(=(colour, orange), 6), Message = b'Traffic light will be orange within 7 time periods of a stop event, not checking the current time period' ;
T = 2, Outcome = later(happ(go), 1), Message = b'Traffic light will get go signal more than 2 time periods after stop signal' ;
T = 2, Outcome = delay(happ(go), 2), Message = b'Traffic light will get go signal exactly 3 time periods after stop signal' ;
T = 2, Outcome = before(=(colour, red), =(colour, green)), Message = b'On stop signal, red light will happen before green light' .
T = 3, Outcome = within(=(colour, green), 3), Message = b'Traffic light will be green within 5 time periods of a stop event' ;
T = 3, Outcome = within(=(colour, orange), 5), Message = b'Traffic light will be orange within 7 time periods of a stop event, not checking the current time period' ;
T = 3, Outcome = later(happ

At time period 2, the light colour is red which fulfils our expectation that the light will be red before it is green.

In [ ]:
?- holdsAt(trafficLight, colour=Colour, 2).
?- happensAt(trafficLight, Event, 2).

Colour = red .
Event = fulf(happ(stop), before(=(colour, red), =(colour, green)), 1, before(=(colour, red), =(colour, green)), dependent, b'On stop signal, red light will happen before green light') .

At time period three, the expectation that the traffic light will get a "go" signal `later` than 2 time periods after the "stop" signal is fulfilled, as a "go" signal event does not occur at time period 2 or 3.

In [ ]:
?- happensAt(trafficLight, Event, 3).

Event = fulf(happ(stop), later(happ(go), 2), 1, later(happ(go), 0), dependent, b'Traffic light will get go signal more than 2 time periods after stop signal') .

At time period 4, the "go" signal occurs. This fulfils the expectations that the "go" signal will ocur 3 time periods after a "stop" signal and that a "stop" signal will preceed a "go" signal by 3 time periods.

In [ ]:
?- happensAt(trafficLight, Event, 4).

Event = go ;
Event = fulf(happ(stop), delay(happ(go), 3), 1, delay(happ(go), 0), dependent, b'Traffic light will get go signal exactly 3 time periods after stop signal') ;
Event = fulf(happ(go), preceded(happ(stop), 3), 4, preceded(happ(stop), 3), dependent, b'On go signal, stop signal will have occurred 3 time periods earlier') .

At time period 5, the light colour becomes green. This fulfils the expectations that the traffic light will be green within 5 time periods of a "stop" signal.

In [ ]:
?- holdsAt(trafficLight, colour=Colour, 5).
?- happensAt(trafficLight, Event, 5).

Colour = green .
Event = fulf(happ(stop), within(=(colour, green), 5), 1, within(=(colour, green), 1), dependent, b'Traffic light will be green within 5 time periods of a stop event') .

At time period 7, the "slow" signal event occurs for the traffic light. This causes the colour of the traffic light to be orange at time period 8, which fulfils our expectation that the traffic light will be orange within 7 time periods of a "stop" signal event, not checking the time period that the "stop" signal occurs in.

In [ ]:
?- T = 7, happensAt(trafficLight, Event, T).
?- T = 8, holdsAt(trafficLight, colour=Colour, T).
?- T = 8, happensAt(trafficLight, Event, T).

T = 7, Event = slow .
T = 8, Colour = orange .
T = 8, Event = fulf(happ(stop), withinStartNext(=(colour, orange), 7), 1, within(=(colour, orange), 0), dependent, b'Traffic light will be orange within 7 time periods of a stop event, not checking the current time period') .

We will reset our environment to change our expectations and show how violations can occur. We retract all of our `initially/1` and `initiates/3` clauses.

In [ ]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
true.
true.

Our traffic light `initiates` clauses and narrative events remain the same, but our expectation rules have changed.

The expectation rules declared are now as follows:
- The traffic light will be green **within** 2 time periods of a stop event.
- The traffic light will be orange within 5 time periods of a stop event, not counting the colour of the light when the stop event occurs (**withinStartNext**).
- The traffic light will only get a go signal more than 4 time periods after a stop signal (expectation that a go signal will only occur **later** than 4 periods after the stop signal).
- The traffic light will get a "go" signal exactly 2 time periods after a stop signal (**delay** expectation)
- When a "stop" signal is received we expect that there will be a green light **before** a red light holds.
- When a "go" signal occurs, it will have been **preceded** by a "stop" signal 2 time periods earlier.

In [ ]:
% File: trafficLightExample.pl
initiates(trafficLight:stop, colour=red, _).
initiates(trafficLight:go, colour=green, _).
initiates(trafficLight:slow, colour=orange).

initially(trafficLight:exp_rule(happ(stop), within(colour=green, 2), dependent, 
"Traffic light will be green within 2 time periods of a stop event")).
initially(trafficLight:exp_rule(happ(stop), withinStartNext(colour=orange, 5), dependent,
"Traffic light will be orange within 5 time periods of a stop event, not checking the current time period")).
initially(trafficLight:exp_rule(happ(stop), later(happ(go), 4), dependent,
"Traffic light will get go signal more than 4 time periods after stop signal")).
initially(trafficLight:exp_rule(happ(stop), delay(happ(go), 2), dependent, 
"Traffic light will get go signal 2 time periods after stop signal")).
initially(trafficLight:exp_rule(happ(stop), before(colour=green, colour=red), dependent, 
"On stop signal, green light will then happen before red light")).
initially(trafficLight:exp_rule(happ(go), preceded(happ(stop),2), dependent,
"On go signal, stop signal will have occurred 2 time periods earlier")).

initially(trafficLight:(colour=orange)).
happensAtNarrative(trafficLight:stop, 1).
happensAtNarrative(trafficLight:go, 4).
happensAtNarrative(trafficLight:slow, 7).

The "stop" signal event at T = 1 triggers five expectations. Note that again, our withinStartNext expectation is not fulfiled at T = 1 even though the light is currently orange. 

In [ ]:
?- run(10).
?- holdsAt(trafficLight, colour=Colour, 1).
?- happensAt(trafficLight, Event, 1).
?- holdsAt(trafficLight, exp(_, _, _, Outcome, _, Message), 1).

true.
Colour = orange .
Event = stop .
Outcome = within(=(colour, green), 2), Message = b'Traffic light will be green within 2 time periods of a stop event' ;
Outcome = withinStartNext(=(colour, orange), 5), Message = b'Traffic light will be orange within 5 time periods of a stop event, not checking the current time period' ;
Outcome = later(happ(go), 4), Message = b'Traffic light will get go signal more than 4 time periods after stop signal' ;
Outcome = delay(happ(go), 2), Message = b'Traffic light will get go signal 2 time periods after stop signal' ;
Outcome = before(=(colour, green), =(colour, red)), Message = b'On stop signal, green light will then happen before red light' .

Again, we see that the countdown value decreases by one for each time period that passes. At time period 2, the traffic light becomes red which violates the expectation that the light will be green before it is red.

In [ ]:
?- holdsAt(trafficLight, exp(_, _, _, Outcome, _, Message), 2).
?- holdsAt(trafficLight, colour=Colour, 2).
?- happensAt(trafficLight, Event, 2).

Outcome = within(=(colour, green), 1), Message = b'Traffic light will be green within 2 time periods of a stop event' ;
Outcome = within(=(colour, orange), 4), Message = b'Traffic light will be orange within 5 time periods of a stop event, not checking the current time period' ;
Outcome = later(happ(go), 3), Message = b'Traffic light will get go signal more than 4 time periods after stop signal' ;
Outcome = delay(happ(go), 1), Message = b'Traffic light will get go signal 2 time periods after stop signal' ;
Outcome = before(=(colour, green), =(colour, red)), Message = b'On stop signal, green light will then happen before red light' .
Colour = red .
Event = viol(happ(stop), before(=(colour, green), =(colour, red)), 1, before(=(colour, green), =(colour, red)), dependent, b'On stop signal, green light will then happen before red light') .

At time period 3, the expectation that the traffic light will be green after 2 time periods of a "stop" signal event is violated, as the colour was not green at time period 2 or 3. The expectation that the traffic light will get a "go" signal event exactly 2 time periods after the stop signal (our `delay` expectation) is also violated.

In [ ]:
?- holdsAt(trafficLight, exp(_, _, _, Outcome, _, Message), 3).
?- happensAt(trafficLight, Event, 3).

Outcome = within(=(colour, green), 0), Message = b'Traffic light will be green within 2 time periods of a stop event' ;
Outcome = within(=(colour, orange), 3), Message = b'Traffic light will be orange within 5 time periods of a stop event, not checking the current time period' ;
Outcome = later(happ(go), 2), Message = b'Traffic light will get go signal more than 4 time periods after stop signal' ;
Outcome = delay(happ(go), 0), Message = b'Traffic light will get go signal 2 time periods after stop signal' .
Event = viol(happ(stop), within(=(colour, green), 2), 1, within(=(colour, green), 0), dependent, b'Traffic light will be green within 2 time periods of a stop event') ;
Event = viol(happ(stop), delay(happ(go), 2), 1, delay(happ(go), 0), dependent, b'Traffic light will get go signal 2 time periods after stop signal') .

At time period 4, the expectation that the traffic light will get a "go" signal more than 4 time periods after a "stop" signal event is violated, as only 3 time periods have occurred since the "stop" signal event. The expectation that a "stop" signal will have preceded a "go" signal by exactly 2 time periods is also violated.

In [ ]:
?- happensAt(trafficLight, Event, 4).

Event = go ;
Event = viol(happ(stop), later(happ(go), 4), 1, later(happ(go), 1), dependent, b'Traffic light will get go signal more than 4 time periods after stop signal') ;
Event = viol(happ(go), preceded(happ(stop), 2), 4, preceded(happ(stop), 2), dependent, b'On go signal, stop signal will have occurred 2 time periods earlier') .

At time period 6, our expectation that the light will become orange within 5 time periods of the "stop" signal, excluding the light colour at T = 1, is also violated.

In [ ]:
?- happensAt(trafficLight, Event, 6).

Event = viol(happ(stop), withinStartNext(=(colour, orange), 5), 1, within(=(colour, orange), 0), dependent, b'Traffic light will be orange within 5 time periods of a stop event, not checking the current time period') .

### Adjustable and independent expectations

These two changes to expectations are explained in the [Expectations notebook](https://github.com/katetruman/MultiAgentEC/blob/master/Expectations.ipynb). As the vast majority of expectations are likely to be dependent, it could be useful to allow a shorthand version of expectation rules without a status specified, although this would increase the already large number of different versions of `exp` fluents.

Note that the `progress/2` fluent in **dec.pl** is now `progress/4`. The arguments are the **Actor**, **Residual Expectation**, **Progressed Expectation** and the **time period**. Specifying the time period allows us to create adjustable expectations based on other fluents.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>